In [1]:
import cvxpy as cp
import numpy as np
import plotly.graph_objects as go

# Gradient Descent and Newton's Method Optimization Example

Find the minimum value of 

$$f(x) = - \displaystyle\sum_{i=1}^m \log\left(1-a_i^Tx \right) - \displaystyle\sum_{i=1}^n \log(1-x_i^2)$$

where $x \in \mathbb{R}^n$. 

In [2]:
# generate problem data
m=200
n=100
A = np.random.normal(0,1,(200,100))

In [3]:
def f(x):
    return -1*np.sum(np.log(1-A@x))-np.sum(np.log(1-x**2))

In [4]:
def grad(x):
    return A.T@(1/(1-A@x))+1/(1-x) - 1/(1+x)

In [5]:
def neg_grad(x):
    return -1*(A.T@(1/(1-A@x))+1/(1-x) - 1/(1+x))

In [6]:
def steepest_descent(x):
    g = grad(x)
    j = np.argmax(np.abs(g))
    sd = np.zeros(len(g))
    if g[j]>=0:
        sd[j] = -1.0
    else:
        sd[j] = 1.0
    return sd

In [7]:
def back_track(x, objective, alpha, beta, descent_direction):
    t = 1
    while np.max(A@(x+t*descent_direction(x)))>=1 or np.max(np.abs(x+t*descent_direction(x)))>=1:
        t = beta*t
    while objective(x + t*descent_direction(x)) > objective(x)+t*alpha*(grad(x)@descent_direction(x)):
        t = beta*t
    return t

In [8]:
# Gradient Descent
x = np.zeros(100) # initial point
grad_tol = 1e-3
alpha = 0.01
beta = 0.5
i = 0
gd_objective_values = []
while i < 1000:
    if np.sqrt(grad(x)@grad(x)) <= grad_tol:
        break
    else:
        t = back_track(x, f, alpha, beta, neg_grad)
        x = x + t*neg_grad(x)
        gd_objective_values.append(f(x))
    i += 1
gd_iters = i    
print('Opt objective value is', np.min(gd_objective_values))

Opt objective value is -173.84754285782384


In [9]:
# l1 steepest descent
x = np.zeros(100) # initial point
tol = 1e-3
alpha = 0.01
beta = 0.5
i = 0
sd_objective_values = []
while i < 1000:
    if np.sum(np.abs(grad(x))) <= tol:
        break
    else:
        t = back_track(x, f, alpha, beta, steepest_descent)
        x = x + t*steepest_descent(x)
        sd_objective_values.append(f(x))
    i += 1
sd_iters = i
print('Opt objective value is', np.min(sd_objective_values))

Opt objective value is -173.84664466227022


In [10]:
fig = go.Figure(data=go.Scatter(x=np.arange(1,gd_iters+1), y=gd_objective_values, name = 'Gradient Descent',mode='lines'))
fig.add_trace(go.Scatter(x=np.arange(1,sd_iters+1), y=sd_objective_values, name='Steepest Descent', 
                         mode='lines', line=dict(color='green')))
fig.update_layout(
    autosize=False,
    width=800,
    height=400)
fig.update_layout(xaxis=dict(range=[-10,i]))
fig.show()

In [11]:
z = cp.Variable(100)
objective = cp.Minimize(-1*cp.sum(cp.log(1-A@x))-cp.sum(cp.log(1-x**2)))
constraints = []
prob = cp.Problem(objective, constraints)
prob.solve()

-173.84664466227022

In [12]:
opt_gap = np.array(gd_objective_values) - np.min(gd_objective_values)
fig=go.Figure(go.Scatter(x=np.arange(1,gd_iters+1), y=opt_gap, mode='lines', line=dict(color='green')))
fig.update_layout(
    autosize=False,
    width=800,
    height=400)
fig.update_layout(xaxis=dict(range=[-10,i]))
fig.show()

In [13]:
def hessian(x):
    return A.T@np.diag(1/((1-A@x)**2))@A + np.diag(1/((1-x)**2)) + np.diag(1/((1+x)**2))
    

In [14]:
def newton_step(x):
    g = grad(x)
    H = hessian(x)
    H_inv = np.linalg.inv(H)
    return -1*H_inv@g
def newton_dec(x):
    g = grad(x)
    H = hessian(x)
    H_inv = np.linalg.inv(H)
    return g@(H_inv@g)

In [15]:
# Newtons Method
x = np.zeros(100) # initial point
tol = 1e-3
alpha = 0.01
beta = 0.5
i = 0
dec = np.inf
nm_objective_values = []
while i < 1000:
    if dec <= tol:
        break
    else:
        t = back_track(x, f, alpha, beta, newton_step)
        x = x + t*newton_step(x)
        dec = -grad(x)@newton_step(x)
        nm_objective_values.append(f(x))
    i += 1
nm_iters = i
print('Opt objective value is', np.min(nm_objective_values))
print('Number of iterations:', nm_iters)

Opt objective value is -173.8470909704829
Number of iterations: 6


In [16]:
fig = go.Figure(data=go.Scatter(x=np.arange(1,nm_iters+1), y=nm_objective_values, mode='lines'))
fig.show()